In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath("../../mathematical_models"))
from mathematical_models.s_on_f import ScalarOnFunctionModel
from mathematical_models.f_on_f import FunctionOnFunctionModel
from mathematical_models.s_on_s import ScalarOnScalarModel

sys.path.append(os.path.abspath("../../optimizers"))
from optimizers.nbdo import NBDO

sys.path.append(os.path.abspath("../../utilities"))
from utilities.plotting.plot_fun import subplot_results

### Settings

In [2]:
N = 20 # runs
Kx = [[16]]
epochs = 100
s_on_s_model = ScalarOnScalarModel(Kx=Kx[0])

In [3]:
optimizer_s_on_s = NBDO(model=s_on_s_model, latent_dim=4)
optimizer_s_on_s.compute_train_set(num_designs=5_000, runs=N)
history = optimizer_s_on_s.fit(epochs=epochs)

Epoch 1/100


2024-08-16 21:06:29.839516: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


125/125 [==============================] - 1s 3ms/step - loss: 57171.8711 - val_loss: 3.0850
Epoch 2/100
125/125 [==============================] - 0s 2ms/step - loss: 1.3392 - val_loss: 0.9714
Epoch 3/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9680 - val_loss: 0.9657
Epoch 4/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9612 - val_loss: 0.9609
Epoch 5/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9607 - val_loss: 0.9606
Epoch 6/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9601 - val_loss: 0.9600
Epoch 7/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9599 - val_loss: 0.9599
Epoch 8/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9598 - val_loss: 0.9598
Epoch 9/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9598 - val_loss: 0.9598
Epoch 10/100
125/125 [==============================] - 0s 2ms/step - loss: 0.9598 - val_loss: 0.95

In [5]:
best_cr, best_des = optimizer_s_on_s.optimize()

Iteration No: 1 started. Evaluating function at random point.
1/1 [==============================] - 0s 84ms/step
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1040
Function value obtained: 0.9511
Current minimum: 0.9511
Iteration No: 2 started. Evaluating function at random point.
1/1 [==============================] - 0s 9ms/step
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0222
Function value obtained: 0.9652
Current minimum: 0.9511
Iteration No: 3 started. Evaluating function at random point.
1/1 [==============================] - 0s 9ms/step
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0233
Function value obtained: 0.9797
Current minimum: 0.9511
Iteration No: 4 started. Evaluating function at random point.
1/1 [==============================] - 0s 9ms/step
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0252
Function value obtained: 0.9512
Current minimum: 0.9511
Iteration No: 5 started

In [6]:
print(np.round(best_des, 3))

[[ 1.    -1.    -0.502 -1.    -1.     1.     1.    -1.     1.     1.
  -1.     1.     1.     1.     1.     1.   ]
 [-1.    -1.     1.     1.    -1.    -1.    -1.    -1.    -1.    -1.
  -1.    -1.     0.766  1.    -1.    -1.   ]
 [ 1.    -1.     1.     1.     1.    -1.     1.     1.    -1.     1.
   1.    -1.    -1.     1.     1.    -1.   ]
 [-1.     1.    -1.    -1.     1.    -1.    -1.    -1.     1.     1.
   1.    -1.    -1.     1.    -1.     1.   ]
 [ 1.     1.    -1.     1.     1.    -1.    -1.     1.     1.     1.
  -1.     1.     1.     1.    -1.    -1.   ]
 [-1.    -1.    -1.     1.     1.     1.     1.    -1.    -1.     1.
   0.93   1.    -1.    -1.     1.    -1.   ]
 [ 1.     1.     1.     1.     1.     1.    -1.     1.    -1.    -1.
   1.     1.     1.    -1.    -0.329  1.   ]
 [-1.     1.     1.     1.    -1.     1.    -1.     1.    -1.     1.
  -1.     1.    -1.     1.     1.     1.   ]
 [-1.    -1.    -1.     1.    -1.    -0.348 -1.     1.     1.     1.
   1.    -1.     1.

In [7]:
best_cr

0.9510582568414039